<a href="https://colab.research.google.com/github/JorgeAccardi/auscultacion-presa/blob/main/Visualizaci%C3%B3n_Inclin%C3%B3metros_GKN_Versi%C3%B3n_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import io
from IPython.display import display, clear_output
import ipywidgets as widgets

# Diccionarios para almacenar por tipo de archivo
instrumentos = [
    "puntos_fijos_mi",
    "puntos_fijos_md",
    "inclinometros",
    "asentamiento",
    "piezometros_electricos",
    "piezometros_casagrande",
    "freatimetros",
    "extensometro"
]

datos_csv = {inst: pd.DataFrame() for inst in instrumentos}
datos_xlsx = {inst: pd.DataFrame() for inst in instrumentos}

# Función para detectar tipo de instrumento por nombre
def detectar_instrumento(nombre):
    nombre = nombre.lower()
    if "puntosfijos" in nombre or "pf" in nombre:
        if "mi" in nombre:
            return "puntos_fijos_mi"
        elif "md" in nombre:
            return "puntos_fijos_md"
        else:
            return None  # Puntos fijos sin margen, no válido
    elif "incli" in nombre:
        return "inclinometros"
    elif "as" in nombre:
        return "asentamiento"
    elif "pe" in nombre:
        return "piezometros_electricos"
    elif "pcg" in nombre:
        return "piezometros_casagrande"
    elif "frea" in nombre:
        return "freatimetros"
    elif "ext" in nombre:
        return "extensometro"
    return None

# --- Widget de carga de archivos ---
upload_widget = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=True,
    description='Subir archivos',
    style={'button_color': 'lightblue'}
)

output = widgets.Output()

# Función principal de carga
def cargar_archivos(change):
    with output:
        clear_output(wait=True)
        archivos = upload_widget.value

        if not archivos:
            print("⚠️ No se subió ningún archivo.")
            return

        for nombre_archivo, archivo_info in archivos.items():
            try:
                contenido = archivo_info['content']
                extension = nombre_archivo.split('.')[-1].lower()
                instrumento = detectar_instrumento(nombre_archivo)

                if not instrumento:
                    print(f"❌ Instrumento no reconocido o mal nombrado: {nombre_archivo}")
                    continue

                # Cargar el archivo
                if extension == 'csv':
                    df = pd.read_csv(io.BytesIO(contenido), encoding='utf-8')
                    datos_csv[instrumento] = pd.concat([datos_csv[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (CSV)")
                elif extension == 'xlsx':
                    df = pd.read_excel(io.BytesIO(contenido))
                    datos_xlsx[instrumento] = pd.concat([datos_xlsx[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (XLSX)")
                else:
                    print(f"❌ Formato no compatible: {nombre_archivo}")
            except Exception as e:
                print(f"❌ Error al procesar {nombre_archivo}: {e}")

        mostrar_menu()

# Función de visualización dinámica
def mostrar_menu():
    opciones = []
    for origen in ['csv', 'xlsx']:
        for instrumento in instrumentos:
            opciones.append(f"{instrumento} ({origen})")

    selector = widgets.Dropdown(
        options=opciones,
        description='Seleccionar DataFrame:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='60%')
    )

    def mostrar_datos(change):
        clear_output(wait=True)
        display(upload_widget, output)
        seleccion = selector.value
        instrumento, origen = seleccion.split(" ")
        instrumento = instrumento.strip()
        origen = origen.strip("()")

        print(f"📊 Mostrando: {instrumento.upper()} ({origen.upper()})")
        if origen == "csv":
            display(datos_csv[instrumento].head())
        else:
            display(datos_xlsx[instrumento].head())

    selector.observe(mostrar_datos, names='value')
    display(selector)

# Conectar evento
upload_widget.observe(cargar_archivos, names='value')

# Mostrar interfaz
display(upload_widget)
display(output)

FileUpload(value={}, accept='.csv,.xlsx', description='Subir archivos', multiple=True, style=ButtonStyle(butto…

Output()

In [2]:
# 💡 Esto instalará versiones compatibles
!pip install -U plotly==6.1.1 kaleido==0.2.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 12.2 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1


In [2]:
# === VISUALIZACIÓN INCLINÓMETROS CON SELECTOR DE ORIGEN, INCLINÓMETRO Y PROFUNDIDAD ===

import pandas as pd

try:
    import plotly.graph_objects as go
    import plotly.colors as pc
    import ipywidgets as widgets
    from IPython.display import display, clear_output, HTML
    import importlib.util
    import os
except ImportError:
    raise ImportError("Ejecuta: pip install plotly ipywidgets")

# --- Widgets y salidas ---
origen_dropdown = widgets.Dropdown(options=["CSV", "XLSX"], value="CSV", description="Origen:")
inclinometro_dropdown = widgets.Dropdown(description="Inclinómetro:")
profundidad_dropdown = widgets.Dropdown(description="Profundidad:")
variable_dropdown = widgets.Dropdown(description="Variable:")
anio_dropdown = widgets.Dropdown(description="Año:")
nro_sonda_dropdown = widgets.Dropdown(description="Nro Sonda:")
estilo_dropdown = widgets.Dropdown(
    options=[
        "Curvas suaves (spline)", "Líneas rectas", "Puntos",
        "Líneas + Puntos", "Área apilada", "Área + Líneas", "Área + Líneas + Puntos"
    ],
    value="Curvas suaves (spline)",
    description="Estilo:"
)

tamanio_dropdown = widgets.Dropdown(
    options={"Pequeño": (600, 400), "Mediano": (900, 500), "Grande": (1200, 700), "Extra grande": (1600, 1000)},
    value=(900, 500),
    description="Tamaño:"
)

grosor_dropdown = widgets.Dropdown(
    options={"Fino (1px)": 1, "Normal (2px)": 2, "Medio (4px)": 4, "Grueso (7px)": 7, "Extra grueso (10px)": 10},
    value=2,
    description="Grosor:"
)

paleta_dropdown = widgets.Dropdown(
    options={
        "Plotly": pc.qualitative.Plotly,
        "D3": pc.qualitative.D3,
        "Viridis": pc.sequential.Viridis,
        "Cividis": pc.sequential.Cividis,
        "Inferno": pc.sequential.Inferno,
        "Pastel": pc.qualitative.Pastel,
        "Bold": pc.qualitative.Bold,
        "Set1": pc.qualitative.Set1,
        "Dark2": pc.qualitative.Dark2
    },
    value=pc.qualitative.Plotly,
    description="Paleta:"
)

output = widgets.Output()
output_guardar = widgets.Output()
boton = widgets.Button(description="Graficar", button_style="success")
boton_guardar = widgets.Button(description="Guardar gráfica", button_style="info")
ruta_text = widgets.Text(value="grafica_inclinometros", description="Ruta y nombre:")
formato_dropdown = widgets.Dropdown(
    options={"PNG": ".png", "JPEG": ".jpg", "SVG": ".svg", "PDF": ".pdf", "HTML": ".html"},
    value=".png",
    description="Formato:"
)

controles_guardar = widgets.HBox([formato_dropdown, ruta_text])

# --- Función para obtener DataFrame según origen ---
def obtener_df(origen):
    try:
        return datos_csv["inclinometros"] if origen == "CSV" else datos_xlsx["inclinometros"]
    except:
        return pd.DataFrame()

# --- Actualizar INCLINÓMETROS disponibles ---
def actualizar_inclinometros(change=None):
    df = obtener_df(origen_dropdown.value)
    if df.empty or 'Inclinometro' not in df.columns:
        inclinometro_dropdown.options = []
        return
    inclinometros = sorted(df['Inclinometro'].dropna().unique())
    inclinometro_dropdown.options = inclinometros
    if inclinometros:
        inclinometro_dropdown.value = inclinometros[0]

# --- Actualizar PROFUNDIDADES disponibles según INCLINÓMETRO ---
def actualizar_profundidades(change=None):
    df = obtener_df(origen_dropdown.value)
    if df.empty or 'Inclinometro' not in df.columns or 'Profundidad' not in df.columns:
        profundidad_dropdown.options = []
        return
    incl = inclinometro_dropdown.value
    if incl:
        profundidades = sorted(df[df['Inclinometro'] == incl]['Profundidad'].dropna().unique())
        profundidad_dropdown.options = ["Todas"] + list(profundidades)
        profundidad_dropdown.value = "Todas"

# --- Actualizar NRO_SONDA disponibles ---
def actualizar_nro_sonda(change=None):
    df = obtener_df(origen_dropdown.value)
    if df.empty or 'Nro_Sonda' not in df.columns:
        nro_sonda_dropdown.options = []
        return
    sondas = sorted(df['Nro_Sonda'].dropna().unique())
    nro_sonda_dropdown.options = ["Todas"] + list(sondas)
    nro_sonda_dropdown.value = "Todas"

# --- Actualizar VARIABLES y AÑOS ---
def actualizar_variables_y_anios(change=None):
    df = obtener_df(origen_dropdown.value)
    if df.empty:
        variable_dropdown.options = []
        anio_dropdown.options = []
        return

    df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True, errors='coerce')
    # Variables disponibles para graficar (A+, A-, B+, B-)
    columnas_excluir = ['Fecha', 'Inclinometro', 'Profundidad', 'Nro_Sonda']
    variables = [c for c in df.select_dtypes(include='number').columns if c not in columnas_excluir]
    # Si no hay variables numéricas detectadas, incluir las esperadas
    if not variables:
        variables = ['A+', 'A-', 'B+', 'B-']

    variable_dropdown.options = variables
    if variables:
        variable_dropdown.value = variables[0]

    anios = sorted(df['Fecha'].dt.year.dropna().unique())
    anio_dropdown.options = ["Todos"] + [str(a) for a in anios]
    anio_dropdown.value = "Todos"

# --- Graficar ---
def graficar(b=None):
    global fig
    with output:
        clear_output(wait=True)
        df = obtener_df(origen_dropdown.value).copy()

        if df.empty:
            print("⚠️ No hay datos disponibles.")
            return

        df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True, errors='coerce')

        incl = inclinometro_dropdown.value
        prof = profundidad_dropdown.value
        variable = variable_dropdown.value
        anio = anio_dropdown.value
        sonda = nro_sonda_dropdown.value
        estilo = estilo_dropdown.value
        ancho, alto = tamanio_dropdown.value
        grosor = grosor_dropdown.value
        paleta = paleta_dropdown.value

        # Filtrar datos
        df_plot = df[df['Inclinometro'] == incl]

        if prof != "Todas":
            df_plot = df_plot[df_plot['Profundidad'] == prof]

        if sonda != "Todas":
            df_plot = df_plot[df_plot['Nro_Sonda'] == sonda]

        if anio != "Todos":
            df_plot = df_plot[df_plot['Fecha'].dt.year == int(anio)]

        df_plot = df_plot.dropna(subset=['Fecha', variable])

        if df_plot.empty:
            print("⚠️ No hay datos para graficar con esa selección.")
            return

        fig = go.Figure()

        # Agrupar por profundidad para crear series diferentes
        if prof == "Todas":
            # Graficar todas las profundidades como series separadas
            profundidades = sorted(df_plot['Profundidad'].unique())
            color_map = {p: paleta[i % len(paleta)] for i, p in enumerate(profundidades)}

            for profundidad in profundidades:
                datos_prof = df_plot[df_plot['Profundidad'] == profundidad]
                line_args = dict(width=grosor, color=color_map[profundidad])
                marker_args = dict(color=color_map[profundidad])

                modo = {
                    "Curvas suaves (spline)": ("lines", "spline"),
                    "Líneas rectas": ("lines", "linear"),
                    "Puntos": ("markers", None),
                    "Líneas + Puntos": ("lines+markers", "linear"),
                    "Área apilada": ("lines", "linear"),
                    "Área + Líneas": ("lines", "linear"),
                    "Área + Líneas + Puntos": ("lines+markers", "linear")
                }
                modo_graf, line_shape = modo[estilo]
                fill = "tozeroy" if "Área" in estilo else None
                stackgroup = "one" if estilo == "Área apilada" else None

                fig.add_trace(go.Scatter(
                    x=datos_prof['Fecha'],
                    y=datos_prof[variable],
                    mode=modo_graf,
                    name=f"Prof. {profundidad}m",
                    line_shape=line_shape,
                    line=line_args,
                    marker=marker_args,
                    fill=fill,
                    stackgroup=stackgroup
                ))
        else:
            # Graficar una sola profundidad
            line_args = dict(width=grosor, color=paleta[0])
            marker_args = dict(color=paleta[0])

            modo = {
                "Curvas suaves (spline)": ("lines", "spline"),
                "Líneas rectas": ("lines", "linear"),
                "Puntos": ("markers", None),
                "Líneas + Puntos": ("lines+markers", "linear"),
                "Área apilada": ("lines", "linear"),
                "Área + Líneas": ("lines", "linear"),
                "Área + Líneas + Puntos": ("lines+markers", "linear")
            }
            modo_graf, line_shape = modo[estilo]
            fill = "tozeroy" if "Área" in estilo else None

            fig.add_trace(go.Scatter(
                x=df_plot['Fecha'],
                y=df_plot[variable],
                mode=modo_graf,
                name=f"{incl} - Prof. {prof}m",
                line_shape=line_shape,
                line=line_args,
                marker=marker_args,
                fill=fill
            ))

        # Configurar layout
        titulo = f"{incl} – {variable}"
        if prof != "Todas":
            titulo += f" (Prof. {prof}m)"
        if sonda != "Todas":
            titulo += f" - Sonda {sonda}"

        fig.update_layout(
            width=ancho, height=alto,
            title=titulo,
            xaxis_title="Fecha",
            yaxis_title=variable,
            legend_title="Profundidad" if prof == "Todas" else "Serie",
            hovermode="x unified"
        )
        fig.show()

# --- Guardar gráfica ---
def guardar_grafica(b=None):
    with output_guardar:
        clear_output(wait=True)
        ext = formato_dropdown.value
        nombre = ruta_text.value
        if not nombre.lower().endswith(ext):
            nombre += ext
        if 'fig' not in globals() or not isinstance(fig, go.Figure):
            print("❌ Generá una gráfica primero.")
            return
        try:
            if ext in [".png", ".jpg", ".svg", ".pdf"]:
                if importlib.util.find_spec("kaleido") is None:
                    print("❌ Instala kaleido:\n%pip install -U kaleido")
                    return
                fig.write_image(nombre, format=ext[1:])
            elif ext == ".html":
                fig.write_html(nombre)
            print(f"✅ Guardado: {os.path.abspath(nombre)}")
        except Exception as e:
            print("❌ Error al guardar:", e)

# --- Eventos ---
origen_dropdown.observe(actualizar_inclinometros, names='value')
inclinometro_dropdown.observe(actualizar_profundidades, names='value')
origen_dropdown.observe(actualizar_variables_y_anios, names='value')
origen_dropdown.observe(actualizar_nro_sonda, names='value')

boton.on_click(graficar)
boton_guardar.on_click(guardar_grafica)

# --- Inicializar y mostrar controles ---
actualizar_inclinometros()
actualizar_variables_y_anios()
actualizar_profundidades()
actualizar_nro_sonda()

display(HTML("<h2 style='color:#1866a3;'>Visualización Interactiva – Inclinómetros</h2>"))
display(origen_dropdown)
display(widgets.HBox([inclinometro_dropdown, profundidad_dropdown]))
display(widgets.HBox([variable_dropdown, nro_sonda_dropdown]))
display(widgets.HBox([anio_dropdown, estilo_dropdown]))
display(widgets.HBox([tamanio_dropdown, grosor_dropdown]))
display(paleta_dropdown)
display(boton)
display(output)
display(controles_guardar)
display(boton_guardar)
display(output_guardar)

Dropdown(description='Origen:', options=('CSV', 'XLSX'), value='CSV')

Dropdown(description='Paleta:', options={'Plotly': ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#1…

Button(button_style='success', description='Graficar', style=ButtonStyle())

Output()

Button(button_style='info', description='Guardar gráfica', style=ButtonStyle())

Output()

# 📈 Visualización Interactiva – Inclinómetros

Este panel permite visualizar los datos de inclinómetros según la estructura de columnas:

- `Fecha`: Fecha de la medición
- `Inclinometro`: Nombre del inclinómetro
- `Profundidad`: Profundidad del registro
- `A+`, `A-`, `B+`, `B-`: Lecturas en los ejes del inclinómetro
- `Nro_Sonda`: Número de sondaje (opcional)

### 🎛️ Controles disponibles:

- **Origen**: Permite seleccionar si los datos se cargaron desde archivos `CSV` o `XLSX`.
- **Inclinómetro**: Permite elegir el instrumento a visualizar.
- **Año**: Filtra los registros por año (o todos).
- **Eje**: Selecciona el eje de deformación a graficar (`A+`, `A-`, `B+`, `B-`).
- **Tamaño**: Tamaño de la gráfica (pequeño a extra grande).
- **Paleta**: Paleta de colores a utilizar.
- **Guardar**: Permite guardar la gráfica en distintos formatos (`.png`, `.pdf`, `.svg`, `.html`).

### 📊 Tipo de gráfico:

Se representa el **perfil de deformación** del inclinómetro en función de la profundidad para cada fecha disponible.

- **Eje X**: Deformación (mm)
- **Eje Y**: Profundidad (m), con dirección invertida (0 en superficie)

---

✅ Para utilizar esta visualización, asegurate de tener cargado el DataFrame en `datos_csv["inclinometros"]` o `datos_xlsx["inclinometros"]`.

🧩 Si deseas agregar más filtros (por ejemplo, `Margen` o `Zona`), asegúrate de tener esa columna en tus datos y te ayudo a adaptar el script.



In [4]:
import pandas as pd
import plotly.graph_objects as go
import plotly.colors as pc
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import importlib.util
import os

# === Widgets ===
origen_dropdown = widgets.Dropdown(options=["CSV", "XLSX"], value="CSV", description="Origen:")
inclinometro_dropdown = widgets.Dropdown(description="Inclinómetro:")
anio_dropdown = widgets.Dropdown(description="Año:")
eje_dropdown = widgets.Dropdown(
    options=["A+", "A-", "B+", "B-"],
    value="A+", description="Eje:"
)
estilo_dropdown = widgets.Dropdown(
    options=[
        "Curvas suaves (spline)", "Líneas rectas", "Puntos",
        "Líneas + Puntos", "Área apilada", "Área + Líneas", "Área + Líneas + Puntos"
    ],
    value="Curvas suaves (spline)", description="Estilo:"
)
tamanio_dropdown = widgets.Dropdown(
    options={"Pequeño": (600, 400), "Mediano": (900, 500), "Grande": (1200, 700), "Extra grande": (1600, 1000)},
    value=(900, 500), description="Tamaño:"
)
grosor_dropdown = widgets.Dropdown(
    options={"Fino (1px)": 1, "Normal (2px)": 2, "Medio (4px)": 4, "Grueso (7px)": 7, "Extra grueso (10px)": 10},
    value=2, description="Grosor:"
)
paleta_dropdown = widgets.Dropdown(
    options={
        "Plotly": pc.qualitative.Plotly, "D3": pc.qualitative.D3, "Viridis": pc.sequential.Viridis,
        "Cividis": pc.sequential.Cividis, "Inferno": pc.sequential.Inferno, "Pastel": pc.qualitative.Pastel,
        "Bold": pc.qualitative.Bold, "Set1": pc.qualitative.Set1, "Dark2": pc.qualitative.Dark2
    },
    value=pc.qualitative.Plotly, description="Paleta:"
)

# Botones y salidas
output = widgets.Output()
output_guardar = widgets.Output()
boton = widgets.Button(description="Graficar", button_style="success")
boton_guardar = widgets.Button(description="Guardar gráfica", button_style="info")
ruta_text = widgets.Text(value="grafica_inclinometro", description="Ruta y nombre:")
formato_dropdown = widgets.Dropdown(
    options={"PNG": ".png", "JPEG": ".jpg", "SVG": ".svg", "PDF": ".pdf", "HTML": ".html"},
    value=".png", description="Formato:"
)

# === Funciones auxiliares ===
def obtener_df():
    return datos_csv["inclinometros"] if origen_dropdown.value == "CSV" else datos_xlsx["inclinometros"]

def actualizar_selectores(change=None):
    df = obtener_df()
    if df.empty:
        inclinometro_dropdown.options = []
        anio_dropdown.options = []
        return
    df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True, errors='coerce')
    inclinometro_dropdown.options = sorted(df['Inclinometro'].dropna().unique())
    anios = sorted(df['Fecha'].dt.year.dropna().unique())
    anio_dropdown.options = ["Todos"] + [str(a) for a in anios]

# === Graficar perfil de inclinómetro ===
def graficar(b=None):
    global fig
    with output:
        clear_output(wait=True)
        df = obtener_df().copy()
        if df.empty:
            print("⚠️ No hay datos disponibles.")
            return

        df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True, errors='coerce')
        eje = eje_dropdown.value
        inc = inclinometro_dropdown.value
        anio = anio_dropdown.value

        df = df[df['Inclinometro'] == inc]
        if anio != "Todos":
            df = df[df['Fecha'].dt.year == int(anio)]

        if df.empty:
            print("⚠️ No hay datos para graficar con esa selección.")
            return

        fig = go.Figure()
        fechas = sorted(df['Fecha'].dropna().unique())
        paleta = paleta_dropdown.value
        modo = {
            "Curvas suaves (spline)": ("lines", "spline"),
            "Líneas rectas": ("lines", "linear"),
            "Puntos": ("markers", None),
            "Líneas + Puntos": ("lines+markers", "linear"),
            "Área apilada": ("lines", "linear"),
            "Área + Líneas": ("lines", "linear"),
            "Área + Líneas + Puntos": ("lines+markers", "linear")
        }
        modo_graf, line_shape = modo[estilo_dropdown.value]
        fill = "tozeroy" if "Área" in estilo_dropdown.value else None
        stackgroup = "one" if estilo_dropdown.value == "Área apilada" else None

        for i, fecha in enumerate(fechas):
            d = df[df['Fecha'] == fecha].copy()
            if eje not in d.columns or 'Profundidad' not in d.columns:
                continue
            d = d.dropna(subset=[eje, 'Profundidad'])
            if d.empty:
                continue
            fig.add_trace(go.Scatter(
                x=d[eje], y=d['Profundidad'],
                mode=modo_graf,
                name=str(fecha.date()),
                line=dict(width=grosor_dropdown.value, color=paleta[i % len(paleta)]),
                marker=dict(color=paleta[i % len(paleta)]),
                fill=fill, stackgroup=stackgroup,
                line_shape=line_shape
            ))

        ancho, alto = tamanio_dropdown.value
        fig.update_layout(
            width=ancho, height=alto,
            title=f"{inc} – Perfil {eje}",
            xaxis_title=f"Deformación ({eje})",
            yaxis_title="Profundidad (m)",
            yaxis_autorange="reversed",
            hovermode="closest",
            legend_title="Fecha"
        )
        fig.show()

# === Guardar ===
def guardar_grafica(b=None):
    with output_guardar:
        clear_output(wait=True)
        ext = formato_dropdown.value
        ruta = ruta_text.value
        if not ruta.endswith(ext):
            ruta += ext
        if 'fig' not in globals():
            print("❌ Primero debes generar la gráfica.")
            return
        try:
            if ext in [".png", ".jpg", ".svg", ".pdf"]:
                if importlib.util.find_spec("kaleido") is None:
                    print("❌ Instala 'kaleido':\n%pip install -U kaleido")
                    return
                fig.write_image(ruta)
            elif ext == ".html":
                fig.write_html(ruta)
            print(f"✅ Guardado: {os.path.abspath(ruta)}")
        except Exception as e:
            print("❌ Error:", e)

# === Eventos ===
origen_dropdown.observe(actualizar_selectores, names='value')
boton.on_click(graficar)
boton_guardar.on_click(guardar_grafica)

# === Mostrar ===
actualizar_selectores()

display(HTML("<h2 style='color:#1866a3;'>Visualización Interactiva – Inclinómetros</h2>"))
display(widgets.HBox([origen_dropdown, inclinometro_dropdown]))
display(widgets.HBox([anio_dropdown, eje_dropdown]))
display(widgets.HBox([estilo_dropdown, tamanio_dropdown]))
display(widgets.HBox([grosor_dropdown, paleta_dropdown]))
display(boton)
display(output)
display(widgets.HBox([formato_dropdown, ruta_text, boton_guardar]))
display(output_guardar)


Button(button_style='success', description='Graficar', style=ButtonStyle())

Output()

Output()